### Import Libs

In [1]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

### Spark Session

In [2]:
spark = (SparkSession.builder
                     .master("local[*]")
                     .appName("desafioMod3")
                     .getOrCreate())
spark.version

'3.3.1'

In [3]:
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [4]:
def displayFunc(df, rows = None):
    df = df.to_pandas_on_spark()
    if rows == None:
        dfFim = df.head()
    else:
        dfFim = df.head(rows)
    return dfFim

In [7]:
def loadFile(fileTxt):
      pathTxt = f'C:/Lenzi/Big Data - Analytics/IGTI-BOOTCAMP/Modulo-3/Desafio-Mod3/microdados_Enade_2017_LGPD/2.DADOS/'

      df = (
             spark.read
                  .format("csv")
                  .option("header", "true")
                  .option("sep", ";")
                  .option("inferSchema", "true")
                  .load(f'{pathTxt}{fileTxt}')
            )
      return df

In [15]:
## Load Files
df_enade       = loadFile('microdados2017_arq1.txt')
df_enade_media = loadFile('microdados2017_arq3.txt')
df_enade_sexo  = loadFile('microdados2017_arq5.txt')
df_enade_idade  = loadFile('microdados2017_arq6.txt')
df_enade_estrangeiro  = loadFile('microdados2017_arq9.txt')

## create Views
df_enade.createOrReplaceTempView("vw_enade")
df_enade_sexo.createOrReplaceTempView("vw_enade_sexo")
df_enade_media.createOrReplaceTempView("vw_enade_media")
df_enade_idade.createOrReplaceTempView("vw_enade_idade")
df_enade_estrangeiro.createOrReplaceTempView("vw_enade_estrangeiro")

In [9]:
## 01. Qual era o número de alunos de cursos de Filosofia inscritos no ENADE 2017? 
## a. 2596. b. 3018. c. 7113. d. 4615.
## R: C

df_enade.filter(" CO_GRUPO IN ('3201', '3202') ").count()

7113

In [10]:
## 02. Qual é o número de alunos dos cursos de Filosofia do sexo Masculino? 
## a. 5739. b. 4532. c. 4286. d. 4783.
## R: B

spark.sql("""
               SELECT TP_SEXO,
                      COUNT(CO_CURSO) AS TOTAL
                 FROM vw_enade_sexo 
                WHERE CO_CURSO IN (SELECT CO_CURSO 
                                     FROM vw_enade  
                                    WHERE CO_GRUPO IN(3201,3202)) 
                  AND TP_SEXO IS NOT NULL                                    
                  -- AND TP_SEXO = 'M'  
                GROUP BY TP_SEXO
          """).show()

+-------+-----+
|TP_SEXO|TOTAL|
+-------+-----+
|      F| 2580|
|      M| 4532|
+-------+-----+



In [11]:
## 03. Qual é o código de UF que possui a maior média de nota geral 
##     entre os alunos dos cursos de Filosofia?
## a. 31. b. 53. c. 33. d. 42.
## R: B

spark.sql("""
              SELECT e.CO_UF_CURSO AS CO_UF_CURSO,
                     ROUND(AVG(m.NT_GER),2) AS NT_GER
                FROM vw_enade_media AS m
               INNER JOIN vw_enade AS e
                  ON m.CO_CURSO == e.CO_CURSO
                 AND m.NU_ANO == e.NU_ANO
               WHERE e.CO_GRUPO IN (3201,3202)
                 AND m.NT_GER IS NOT NULL
               GROUP BY e.CO_UF_CURSO
               ORDER BY NT_GER DESC
          """).show(1)

+-----------+------+
|CO_UF_CURSO|NT_GER|
+-----------+------+
|         53| 54.25|
+-----------+------+
only showing top 1 row



In [12]:
## 04. Qual é a diferença das médias de idade (arredondado para 2 dígitos decimais) 
##     entre os alunos dos cursos de filosofia e física?
## a. 3,45. b. 0,50. c. 7,22. d. 5,57.
## R: D

spark.sql("""
            WITH 
            filosofia
                   AS (
                        SELECT ROUND(AVG(NU_IDADE),2) AS MEDIA_NU_IDADE,
                               'filosofia' AS CURSO
                          FROM vw_enade_idade 
                         WHERE CO_CURSO IN (SELECT CO_CURSO 
                                              FROM vw_enade  
                                             WHERE CO_GRUPO IN(3201,3202))
                      ),
            fisica
                AS (
                     SELECT ROUND(AVG(NU_IDADE),2) AS MEDIA_NU_IDADE,
                            'fisica' AS CURSO
                       FROM vw_enade_idade 
                      WHERE CO_CURSO IN (SELECT CO_CURSO 
                                           FROM vw_enade  
                                          WHERE CO_GRUPO IN(1401,1402))
                  )
                
                 SELECT MEDIA_NU_IDADE 
                        - 
                        (SELECT MEDIA_NU_IDADE
                           FROM fisica) AS DIF_MEDIA_IDADE
                   FROM filosofia 
                
          """).show()

+---------------+
|DIF_MEDIA_IDADE|
+---------------+
|           5.57|
+---------------+



In [ ]:
## 05. Qual é a maior nota geral entre alunos de filosofia que consideraram a prova “Fácil” 
##     na parte de Componente Específico ?
## a. 93. b. 96. c. 87. d. 89.
## R: D

spark.sql("""
            SELECT MAX(m.NT_GER) AS NT_GERAL
              FROM vw_enade_media AS m
             INNER JOIN vw_enade AS e
                ON m.CO_CURSO == e.CO_CURSO
               AND m.NU_ANO == e.NU_ANO
             WHERE e.CO_GRUPO IN (3201,3202)
               AND m.CO_RS_I1 == 'B'
          """).show()

+--------+
|NT_GERAL|
+--------+
|    89.5|
+--------+



In [ ]:
## 06. Qual é a média da nota geral para alunos de filosofia, cujo curso está no estado (UF) de código 43?
## a. 56,32. b. 46,97. c. 47,11. d. 49,34.
## R: ??

spark.sql("""
              SELECT ROUND(AVG(NT_GER),2) AS NT_GERAL
                FROM vw_enade_media
               WHERE CO_CURSO IN (SELECT CO_CURSO 
                                    FROM vw_enade  
                                   WHERE CO_GRUPO IN (3201,3202)
                                     AND CO_UF_CURSO == 43)
                AND NT_GER IS NOT NULL
         """).show()

+--------+
|NT_GERAL|
+--------+
|   47.56|
+--------+



In [13]:
##06 Qual é a média da nota geral para alunos de filosofia, cujo curso está no estado (UF) de código43?
#a. 56,32. b. 46,97. c. 47,11. d. 49,34.
spark.sql("""
            SELECT ROUND(AVG(m.NT_GER),2) AS NT_GERAL
              FROM vw_enade_media AS m
             INNER JOIN vw_enade AS e
                ON m.CO_CURSO == e.CO_CURSO
               AND m.NU_ANO == e.NU_ANO
             WHERE e.CO_GRUPO IN (3201,3202)
               AND e.CO_UF_CURSO == 43
          """).show()

+--------+
|NT_GERAL|
+--------+
|   45.88|
+--------+



In [57]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

w = Window.partitionBy("NU_ANO").orderBy("CO_CURSO")

df_enade_row =  df_enade.withColumn("columnindex", row_number().over(w))
df_enade_media_row =  df_enade_media.withColumn("columnindex", row_number().over(w))

newDF = df_enade_row.join(df_enade_media_row, df_enade_row.columnindex == df_enade_media_row.columnindex, 'left')
newDF.drop("df_enade_media_row.columnindex","df_enade_media_row.NU_ANO","df_enade_media_row.CO_CURSO")
newDF.createOrReplaceTempView("vw_quest6")
newDF.count()

537358

In [62]:
spark.sql('''select CO_UF_CURSO, avg(NT_GER) as avg_media from vw_quest6
where CO_GRUPO IN (3201,3202) and CO_UF_CURSO == 43
group by CO_UF_CURSO
order by avg_media desc''').show(1)

+-----------+------------------+
|CO_UF_CURSO|         avg_media|
+-----------+------------------+
|         43|47.561388888888914|
+-----------+------------------+



In [14]:
## 07. Qual é o estado brasileiro que possui a maior quantidade de alunos de nacionalidade estrangeira inscritos no ENADE 2017?
#a. São Paulo. b. Minas Gerais. c. Rio de Janeiro.d. Paraná.

In [17]:
spark.sql("""
             WITH estados
               AS (  
                    SELECT CO_CURSO,
                           CO_UF_CURSO,
                           CASE 
                                WHEN CO_UF_CURSO = '35' THEN 'São Paulo (SP)'
                                WHEN CO_UF_CURSO = '31' THEN 'Minas gerais (MG)'
                                WHEN CO_UF_CURSO = '33' THEN 'Rio de Janeiro (RJ)'
                                WHEN CO_UF_CURSO = '41' THEN 'Paraná (PR)'
                           ELSE '* SEM NOME *' 
                           END AS ESTADO
                         FROM vw_enade
                    )

             SELECT ESTADO,
                    count(ESTADO) AS TOTAL
               FROM estados    
              WHERE CO_CURSO IN (SELECT CO_CURSO
                                   FROM vw_enade_estrangeiro
                                  WHERE QE_I03 == 'C')
                AND ESTADO <> '* SEM NOME *'   
              GROUP BY ESTADO
              ORDER BY TOTAL DESC
          """).show()

+-------------------+-----+
|             ESTADO|TOTAL|
+-------------------+-----+
|     São Paulo (SP)|40438|
|        Paraná (PR)|31169|
|Rio de Janeiro (RJ)|12709|
|  Minas gerais (MG)| 7757|
+-------------------+-----+



In [ ]:
#08. Qual é o código do estado brasileiro que possui a maior média de idade de alunos de filosofia?
#a. 32.b. 15.c. 35.d. 21.


In [18]:
spark.sql("""
             SELECT ROUND(AVG(ei.NU_IDADE),2) AS MED_IDADE,
                    e.CO_UF_CURSO
               FROM vw_enade AS e
              INNER JOIN vw_enade_idade AS ei
                 ON e.CO_CURSO == ei.CO_CURSO
                AND e.NU_ANO == ei.NU_ANO
              WHERE e.CO_GRUPO IN (3201,3202)                  
              GROUP BY e.CO_UF_CURSO
              ORDER BY MED_IDADE DESC
          """).show()

+---------+-----------+
|MED_IDADE|CO_UF_CURSO|
+---------+-----------+
|    41.04|         42|
|    39.37|         21|
|    38.42|         11|
|    38.01|         32|
|    36.59|         43|
|    36.39|         31|
|    36.31|         15|
|     35.9|         41|
|    35.71|         35|
|    35.56|         29|
|    34.79|         12|
|    34.35|         33|
|    33.92|         51|
|    33.72|         50|
|    33.22|         25|
|    33.18|         53|
|    33.05|         27|
|    32.84|         22|
|    32.59|         52|
|    32.52|         17|
+---------+-----------+
only showing top 20 rows



In [42]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

w = Window.partitionBy("NU_ANO").orderBy("CO_CURSO")

df_enade_row =  df_enade.withColumn("columnindex", row_number().over(w))
df_enade_idade_row =  df_enade_idade.withColumn("columnindex", row_number().over(w))

newDF = df_enade_row.join(df_enade_idade_row, df_enade_row.columnindex == df_enade_idade_row.columnindex, 'left')
newDF.drop("df_enade_idade_row.columnindex","df_enade_idade_row.NU_ANO","df_enade_idade_row.CO_CURSO")
newDF.createOrReplaceTempView("vw_quest8")
newDF.count()


537358

In [56]:
spark.sql('''select CO_UF_CURSO, round(avg(NU_IDADE)) as avg_idade from vw_quest8
where CO_GRUPO IN (3201,3202)
group by CO_UF_CURSO
order by avg_idade desc''').show(1)

+-----------+---------+
|CO_UF_CURSO|avg_idade|
+-----------+---------+
|         42|     38.0|
+-----------+---------+
only showing top 1 row

